In [1]:
#Import-------------------------------------------------------------------------------
from io import StringIO, BytesIO
import os
import re
from time import sleep
import random
from urllib import request
import requests
import chromedriver_autoinstaller
from webdriver_manager.chrome import ChromeDriverManager
from selenium.common.exceptions import NoSuchElementException
from selenium import webdriver
from selenium.webdriver.chrome.options import Options as ChromeOptions
from selenium.webdriver.firefox.options import Options as FirefoxOptions
from selenium.webdriver.chrome.service import Service
import datetime
import pandas as pd
import platform
from selenium.webdriver.common.keys import Keys

from bs4 import BeautifulSoup

from selenium.webdriver.support.wait import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
import urllib
from urllib.parse import quote

def get_driver(headless=True):
    options = ChromeOptions()
    if headless is True:
        print("Scraping on headless mode.")
        options.add_argument('--headless')
        options.add_argument("--disable-gpu")
        options.add_argument("--no-sandbox")  # An error will occur without this line
        #user_agent = '"Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/114.0.0.0 Safari/537.36"'
        #options.add_argument(f'user-agent={user_agent}')
    else:
        options.headless = False
        #user_agent = '"Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/114.0.0.0 Safari/537.36"'
        #options.add_argument(f'user-agent={user_agent}')
    try:
        driver_path = chromedriver_autoinstaller.install()
        service = Service(executable_path=driver_path)
        # サービスを起動
        driver = webdriver.Chrome(options=options,service=service)
    except Exception as e:
        print('err -> ',e)
        options.binary_location = '/usr/bin/chromium-browser'
        driver = webdriver.Chrome(options=options)
    return driver

driver = get_driver(False)

CHROME >= 115, using mac-arm64 as architecture identifier


In [2]:
from src import const,utils,sqlite
hashtag = '#つのまきあーと'
date = 30

driver = utils.get_driver(False)
url = utils.get_url(hashtag)
driver.get(url)


In [3]:
import datetime
import re

# 入力文字列
#input_string = "11分前"  # または "9月9日(土) 22:28"
def date_parse(input_string:str):
    # 現在の日時を取得
    current_time = datetime.datetime.now()

    if ("秒前" in input_string) and ("昨日" not in input_string):
        # "分前" を抽出して分数に変換
        second_ago = int(input_string.split("秒前")[0])
        
        # 指定された分数だけ過去の日時を計算
        new_time = current_time - datetime.timedelta(seconds=second_ago)

    elif ("分前" in input_string) and ("昨日" not in input_string):
        # "分前" を抽出して分数に変換
        minutes_ago = int(input_string.split("分前")[0])
        
        # 指定された分数だけ過去の日時を計算
        new_time = current_time - datetime.timedelta(minutes=minutes_ago)
    elif "昨日" in input_string:
        output_string = input_string.replace('昨日 ','')
        #　昨日の日時を設定
        time_parts = output_string.split(":")
        new_time = current_time.replace(hour=int(time_parts[0]), minute=int(time_parts[1]), second=0)
        #new_time = new_time + datetime.timedelta(days=-1)

    elif (":" in input_string) and ('月' not in input_string):
        # "16:15" のような場合、今日の日付に指定時刻を設定
        time_parts = input_string.split(":")
        new_time = current_time.replace(hour=int(time_parts[0]), minute=int(time_parts[1]), second=0)
    else:
        # "9月9日(土) 22:28" のような場合、指定されたフォーマットで日時を解析 9月9日(土) 22:28
        output_string = re.sub(r'\([^)]*\)', '', input_string)
        new_time = datetime.datetime.strptime(output_string, "%m月%d日 %H:%M")
        if new_time.month > current_time.month:
            new_time = new_time.replace(year=current_time.year - 1)
        else:
            new_time = new_time.replace(year=current_time.year)
    return new_time

In [ ]:
from bs4 import BeautifulSoup
from tqdm import tqdm
#Import-------------------------------------------------------------------------------
from io import StringIO, BytesIO
import os
import re
from time import sleep
import random
from urllib import request
import requests
import chromedriver_autoinstaller
from webdriver_manager.chrome import ChromeDriverManager
from selenium.common.exceptions import NoSuchElementException
from selenium import webdriver
from selenium.webdriver.chrome.options import Options as ChromeOptions
from selenium.webdriver.firefox.options import Options as FirefoxOptions
from selenium.webdriver.chrome.service import Service
import datetime
import pandas as pd
import platform
from selenium.webdriver.common.keys import Keys

from bs4 import BeautifulSoup

from selenium.webdriver.support.wait import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
import urllib
from urllib.parse import quote

#-------------------------------------------------------------------------------------
# 現在の日時を取得
now = datetime.datetime.now()
date_time = datetime.datetime.now()
tweets = []
load_flag = 3
load_now = 0
while(True):

    #ツイートのリストを取得
    soup = BeautifulSoup(driver.page_source, 'html.parser')
    tweet_links = soup.find_all('div', class_='Tweet_bodyContainer__n_Cs6')#Tweet_bodyContainer__n_Cs6

    #ツイートから各要素を取得

    for tweet in tweet_links:
        url = ''
        try:
            url = [link for link in tweet.find_all('a') if 'pic.twitter.com' in link.text][0].text#ツイートリンク
        except:
            url = ''
        try:
            images = [i['src'] for i in [link for link in tweet.find('div','Tweet_imageContainerWrapper__wPE0R').find_all('img') if 'https://rts-pctr.c.yimg.jp' in link['src']]]#画像のリスト
        except:
            images = []
        like_count = 0#いいねっ
        try:
            like_count = int([a for a in tweet.find_all('a') if 'like' in a.get('href','')][0].find('span').text.replace(',',''))
        except:
            like_count = 0
        user_name = tweet.find('a','Tweet_authorID__B1U8c').text#ユーザーID
        display_name = tweet.find('span','Tweet_authorName__V3waK').text#ユーザー名
        
        date_text = tweet.find('time').text
        date_time = date_parse(date_text)

        if like_count >= 10 and len(images) > 0 and len([t for t in tweets if url == t[0]]) == 0 and url != '':
            print(url,images,date_time,user_name,display_name,like_count)
            tweets.append([
                url,
                images,
                date_time,
                user_name,
                display_name,
                like_count
            ])
    print(f'tweet count -> {len(tweets)}')
    if (now - date_time).days > date:
        print('datetime',date_time)
        print(f'date over => {(now - date_time).days}/{date}')
        break

    #一番下までスクロール
    for ind in range(8):
        driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
        sleep(0.25)
    sleep(3)
    try:
        load_btn = driver.find_element(By.LINK_TEXT,"もっと見る")
        load_btn.click()
    except:
        load_now = load_now + 1
        
    if load_now >= load_flag:
        print(f'button over => {load_now} / {load_flag}')
        break
    

for t in tweets:
    print(t)


pic.twitter.com/ysrXj6IJtO ['https://rts-pctr.c.yimg.jp/BgIFgYJGVIVv8u0nVJvw8B2FauAYMF4jOP5zFJAV4i6hjVpND1OjWZ7RVcmujne-qesEwbaAOnUWC2tPGuxPOxRquaUQ6aLOQlHSJ7wB9ZBSrE35753E6eau56tglX_sNcgWrTtU0AWHO1y60-suZ-7vL0ay8UN7PjJgcVHUK3BU7fXPW9OodWCLPvqXxCQP'] 2024-02-06 00:24:00.840242 @blueta9810 ぶるー太 1761
pic.twitter.com/TZzLzzEuVY ['https://rts-pctr.c.yimg.jp/BgIFgYJGVIVv8u0nVJvw8B2FauAYMF4jOP5zFJAV4i6hjVpND1OjWZ7RVcmujne-N-d6Xm2mGxGTSUZ1_sShw005zRig_HOJLPAJGxpJNheubzRJZhO42-P3CGnEHCiGALfaAXxt-SBwW9wSzCC7VdtpiIUQh8xFjExHtq36rgdPW_tO6UmThdv9VIxJSbMp'] 2024-02-06 18:20:32.840747 @mirurun_az1 みるちぇいす🐏💛 19
pic.twitter.com/BawylWRPhE ['https://rts-pctr.c.yimg.jp/BgIFgYJGVIVv8u0nVJvw8B2FauAYMF4jOP5zFJAV4i6hjVpND1OjWZ7RVcmujne-3-0aJSg38cJIOv0cLzDL-oZhNWe78yiO55W1leQ8zSMPesO_HOFElOh-CeJqI6usrMYqsnarf43t-PPbkR5Ocpk4IqDFcqnNTgcBlsmH9wZ4z4bby99SGpDJ5UcLf7rC', 'https://rts-pctr.c.yimg.jp/BgIFgYJGVIVv8u0nVJvw8B2FauAYMF4jOP5zFJAV4i6hjVpND1OjWZ7RVcmujne-91BP_gAaXULl_aRPfi1qxs4D9Z0BXvT9-24NbjwpuJaH6LH-rsmk0o

In [4]:
input_string = '9月10日(日) 17:01'
output_string = re.sub(r'\([^)]*\)', '', input_string)
print(output_string)

9月10日 17:01


In [20]:
import os
import pandas as pd


base_path = './'
holo_path = os.path.join(base_path, "options", "HoloFanArt.csv")
df = pd.read_csv(holo_path)
records = []
for index, row in df.iterrows():
    r = row
    hashtag = row["hashtag"]
    url = row["url"]
    rec = HoloName(hashtag,url)
    records.append(rec)
print(row)

NameError: name 'HoloName' is not defined

In [16]:
print(row)

url    https://hololive.hololivepro.com/wp-content/up...
Name: #soraArt, dtype: object


In [13]:
print(row['url'])

https://hololive.hololivepro.com/wp-content/uploads/2021/05/tokino_sora_thumb.png


In [19]:
df

,url
hashtag,
#soraArt,https://hololive.hololivepro.com/wp-content/up...
#ロボ子Art,https://hololive.hololivepro.com/wp-content/up...
#miko_Art,https://hololive.hololivepro.com/wp-content/up...
#ほしまちぎゃらりー,https://hololive.hololivepro.com/wp-content/up...
#AZKiART,https://hololive.hololivepro.com/wp-content/up...
...,...
#Artnose,https://hololive.hololivepro.com/wp-content/up...
#ririkart,https://hololive.hololivepro.com/wp-content/up...
#らでん似絵,https://hololive.hololivepro.com/wp-content/up...


In [49]:
#1ページ目
import requests
response = requests.get('https://search.yahoo.co.jp/realtime/api/v1/pagination?crumb=mAPCZQAAAADNdZd0dcgpPQgDqP92PnuPEMSk6lxd4YGq8c2mRAgBCM2UUYvzIQxomlV2QJ6ctst25ElZTfbJIAemJT7h8JGy&p=%23%E3%82%B7%E3%82%AA%E3%83%B3%E3%81%AE%E6%9B%B8%E7%89%A9&rkf=3&b=1&oldestTweetId=1754784896143565172&start=')
response,response.json()['timeline']['entry'][-1]

(<Response [200]>,
 {'id': '1754451529992675610',
  'url': 'https://twitter.com/yoppy_paint/status/1754451529992675610?utm_source=yjrealtime&utm_medium=search',
  'detailUrl': '/realtime/search/tweet/1754451529992675610?detail=1&ifr=tl_twdtl&rkf=1',
  'detailQuoteUrl': '/realtime/search/tweet/1754451529992675610?detail=1&ifr=tl_twdtl&rkf=1&tl=quote',
  'verified': False,
  'displayText': '#さかまた飼育日記 #シオンの書物 シオン先輩の服////♡ https://t.co/PdEsLWNaRx',
  'displayTextBody': '#さかまた飼育日記 #シオンの書物 シオン先輩の服////♡ https://t.co/PdEsLWNaRx',
  'displayTextFragments': '\n \n シオン先輩の服////♡ \n',
  'displayTextEntities': '#さかまた飼育日記\n#シオンの書物\nhttps://t.co/PdEsLWNaRx',
  'urls': [],
  'hashtags': [{'text': 'さかまた飼育日記', 'indices': [0, 9]},
   {'text': 'シオンの書物', 'indices': [10, 17]}],
  'hashtagUrls': {'さかまた飼育日記': '/realtime/search?p=%23%E3%81%95%E3%81%8B%E3%81%BE%E3%81%9F%E9%A3%BC%E8%82%B2%E6%97%A5%E8%A8%98',
   'シオンの書物': '/realtime/search?p=%23%E3%82%B7%E3%82%AA%E3%83%B3%E3%81%AE%E6%9B%B8%E7%89%A9'},
  'mentions'

## ヤフーのリアルタイムのスクレイピング

In [1]:
import requests
import urllib.parse
import datetime
import time

from tqdm import tqdm

from src import const,utils,sqlite

def parse_date(epoch_seconds:int):
    """
    エポック時間をyyyy-mm-ddに変換
    """
    dt = datetime.datetime.utcfromtimestamp(epoch_seconds)
    formatted_date = dt.strftime('%Y-%m-%d')
    return formatted_date


def get_pagination_url(hashtag:str , last_tweet_id=''):
    """
    ハッシュタグと最後のツイートIDを元にツイートを取得
    """
    hashtag_encode = urllib.parse.quote(hashtag)
    base = 'https://search.yahoo.co.jp/realtime/api/v1/pagination?crumb=mAPCZQAAAADNdZd0dcgpPQgDqP92PnuPEMSk6lxd4YGq8c2mRAgBCM2UUYvzIQxomlV2QJ6ctst25ElZTfbJIAemJT7h8JGy'
    url = f'{base}&p={hashtag_encode}&rkf=3&b=1'
    if last_tweet_id != '':
        return url + f'&oldestTweetId={last_tweet_id}&start='
    else:
        return url

def get_tweets(url:str , mode:str):
    """
    URLを元にツイートのリストを取得
    """
    time.sleep(0.5)
    response = requests.get(url)
    if response.status_code != 200:
        raise ValueError(f"response status_code -> {response.status_code} {response}")
    records = []
    for tweet in response.json()['timeline']['entry']:
        id = tweet['id']
        url = tweet['url']
        images = []
        if 'media' in tweet:
            for media in tweet['media']:
                if media['type'] == 'image':
                    image_url = media['item']['mediaUrl']
                    images.append(image_url)
        date = parse_date(tweet['createdAt'])
        user_id = tweet['screenName']
        user_name = tweet['name']
        like_count = tweet['likesCount']

        rec = const.TwitterQueryRecord(
            hashtag,
            mode,
            url,
            date,
            ",".join(images),
            user_id,
            user_name,
            like_count
        )

        if like_count >= 10 and len(images) > 0:
            records.append(rec)
    last_tweet_id = response.json()['timeline']['entry'][-1]['id']
    return records , last_tweet_id

def date_difference(specified_date):
    """
    今日の日付と引数のyyyy-mm-ddを比較して、日数差を返す
    """
    today = datetime.datetime.now().date()
    #specified_date = datetime.datetime.strptime(date_text, "%Y-%m-%d").date()
    difference = abs((today - specified_date).days)
    return difference

def main(hashtag:str , date:int , mode:str):
    tweets = []
    first_url = get_pagination_url(hashtag)
    records , last_tweet_id = get_tweets(first_url , mode)
    for rec in records:
        tweets.append(rec)
    
    index = 0
    while(True):
        url = get_pagination_url(hashtag , last_tweet_id)
        records , last_tweet_id = get_tweets(url , last_tweet_id)
        for rec in records:
            tweets.append(rec)
    
        last_date = records[-1].date
        if date_difference(last_date) > date:
            break
        print(f"\r　date_range -> {date_difference(last_date)}/{date} | tweets -> {len(tweets)}",end="")
    return tweets

In [2]:
hashtag = '#つのまきあーと'
date = 14

In [3]:
tweets = []
first_url = get_pagination_url(hashtag)
records , last_tweet_id = get_tweets(first_url , 'holo')
for rec in records:
    tweets.append(rec)

index = 0
while(True):
    url = get_pagination_url(hashtag , last_tweet_id)
    records , last_tweet_id = get_tweets(url , last_tweet_id)
    for rec in records:
        tweets.append(rec)

    if len(records) > 0:
        last_date = records[-1].date
    if date_difference(last_date) > date:
        break
    print(f"\r　date_range -> {date_difference(last_date)}/{date} | tweets -> {len(tweets)}",end="")

print(f'tweets -> {len(tweets)}')
for tw in tweets:
    print(tw)


　date_range -> 14/14 | tweets -> 249tweets -> 253
Hashtag: #つのまきあーと
Mode: holo
URL: https://twitter.com/nyas0123/status/1755185889066799252?utm_source=yjrealtime&utm_medium=search
Date: 2024-02-07
Images: ['https://rts-pctr.c.yimg.jp/BgIFgYJGVIVv8u0nVJvw8B2FauAYMF4jOP5zFJAV4i6hjVpND1OjWZ7RVcmujne-cNyTkOQb-Ab1VM-jSMhh2SOe9zxcd5XekVQs0Kch6VW8ca8Csp6GBp_nTE8SohNki2xAsxvDlelB5bUwUnwyThrqmPmueHzrHnDN6qMArdFCk4Ehw6lBM7roGOaSzFKz']
User ID: nyas0123
User Name: ささおか
Like Count: 444

Hashtag: #つのまきあーと
Mode: holo
URL: https://twitter.com/shibakarenasu/status/1755176571751866404?utm_source=yjrealtime&utm_medium=search
Date: 2024-02-07
Images: ['https://rts-pctr.c.yimg.jp/BgIFgYJGVIVv8u0nVJvw8B2FauAYMF4jOP5zFJAV4i6hjVpND1OjWZ7RVcmujne-dVXsuOxG13zE87nV4B8krWA1Ct_bq01FS9mho8EkgoRMbJuiM9N90UGR1rQyiYflWuuYpHTVa5EgekC1IONyAIP8NsnDVE6En7zW7ZigZZV0ovFT93Fp6NSnwJrtBpZc']
User ID: shibakarenasu
User Name: しばかれ茄子@skeb募集中
Like Count: 1097

Hashtag: #つのまきあーと
Mode: holo
URL: https://twitter.com/mocimoci001/sta

In [5]:
url

'https://search.yahoo.co.jp/realtime/api/v1/pagination?crumb=mAPCZQAAAADNdZd0dcgpPQgDqP92PnuPEMSk6lxd4YGq8c2mRAgBCM2UUYvzIQxomlV2QJ6ctst25ElZTfbJIAemJT7h8JGy&p=%23%E3%81%A4%E3%81%AE%E3%81%BE%E3%81%8D%E3%81%82%E3%83%BC%E3%81%A8&rkf=3&b=1&oldestTweetId=1752697088062947794&start='

In [15]:
for t in tweets:
    if len(t.images) > 1:
        for i in t.images:
            print(i)
        print('-----------')

https://rts-pctr.c.yimg.jp/BgIFgYJGVIVv8u0nVJvw8B2FauAYMF4jOP5zFJAV4i6hjVpND1OjWZ7RVcmujne-xnXmcO0FCPr_vjxNG6elkgI55o3qiSKY9AhaXomCuzDEr52pHmjHrBSvOY3xf0-imZ8AdF8PcedUmzZW926BaKQAS6S6NmH2evzHrl_XHQQQsllagi-2tfTtDE4g5xPO
https://rts-pctr.c.yimg.jp/BgIFgYJGVIVv8u0nVJvw8B2FauAYMF4jOP5zFJAV4i6hjVpND1OjWZ7RVcmujne-pLwtnQXgGPMckZFVTkPKFXSrvW__LMluT2jeh8gkX7mqN0F0ioS1LGY8pwTeHjPfESgQ8pRECGa9xUsnsz1w-8KXZZ_l9qisR3VfainnTI6KGVrg7mC5AsPbS4i4eMh8
https://rts-pctr.c.yimg.jp/BgIFgYJGVIVv8u0nVJvw8B2FauAYMF4jOP5zFJAV4i6hjVpND1OjWZ7RVcmujne-3TIisnD5OI96eClar072QvQBDJ0DZuP3AWJ7qa_KsIgc927OJyJe2StjemQM_umtKiuUuEKihQLrDptcmuhzunKwa3DIIrz_LgEtq2srqhvZKR4xTvpcu8dF7DX5pGMx
-----------
https://rts-pctr.c.yimg.jp/BgIFgYJGVIVv8u0nVJvw8B2FauAYMF4jOP5zFJAV4i6hjVpND1OjWZ7RVcmujne-IBnhK_nhJcfP4TOV1YAwg5pZFYYmRTvXQA4-uq2ceX1NZH24m_HZo50JhmE3sf6UCkLtTNokm54u3biqLAslbqVoHxgmg5oNldCBqx7OR2LPV4TY1Rdf0OlUFPTc8gi8
https://rts-pctr.c.yimg.jp/BgIFgYJGVIVv8u0nVJvw8B2FauAYMF4jOP5zFJAV4i6hjVpND1OjWZ7RVcmujne-h1R9CxLI4uqppVqK7

### test

In [17]:
from src import utils
driver = utils.get_driver(False)
df = utils.get_tweet('#絵かゆ',3000,7,driver)
df

url -> https://search.yahoo.co.jp/realtime/search?p=%23%E7%B5%B5%E3%81%8B%E3%82%86&ei=UTF-8&mtype=image
tweet count -> 21
tweet count -> 21
tweet count -> 48
tweet count -> 74
tweet count -> 107
tweet count -> 138
tweet count -> 163
tweet count -> 186
tweet count -> 226
tweet count -> 255
datetime 2024-01-29 00:10:00
date over => 8/7
#絵かゆ result : 255


,url,images,date,userId,userName,likeCount
0,pic.twitter.com/0wzLF7x6nL,[https://rts-pctr.c.yimg.jp/BgIFgYJGVIVv8u0nVJ...,2024-02-06 14:19:00.661375,@sYaM_illust,sYaM@絵描き猫,28431
1,pic.twitter.com/laWOBLEXeZ,[https://rts-pctr.c.yimg.jp/BgIFgYJGVIVv8u0nVJ...,2024-02-06 21:04:40.661481,@kajimakoto_sub,カジ マコト@健全,59
2,pic.twitter.com/S69UgEzhYI,[https://rts-pctr.c.yimg.jp/BgIFgYJGVIVv8u0nVJ...,2024-02-06 20:48:40.661581,@yogur_0311,よ〜ぐる,46
3,pic.twitter.com/Mxdvxu0P3Q,[https://rts-pctr.c.yimg.jp/BgIFgYJGVIVv8u0nVJ...,2024-02-06 20:26:40.661673,@hiyori_cosplay,日和みこと🐤🍜,274
4,pic.twitter.com/VGNxYy0shL,[https://rts-pctr.c.yimg.jp/BgIFgYJGVIVv8u0nVJ...,2024-02-06 20:10:40.661956,@hau___maru,はぅ。,36
...,...,...,...,...,...,...
250,pic.twitter.com/rAo2AFYdVg,[https://rts-pctr.c.yimg.jp/BgIFgYJGVIVv8u0nVJ...,2024-01-29 10:07:00.000000,@Shi_a_ihS,しお,121
251,pic.twitter.com/ko5KG3myk9,[https://rts-pctr.c.yimg.jp/BgIFgYJGVIVv8u0nVJ...,2024-01-29 06:00:00.000000,@Kulibel_,くりべる🍙,52
252,pic.twitter.com/aToXSTCfsx,[https://rts-pctr.c.yimg.jp/BgIFgYJGVIVv8u0nVJ...,2024-01-29 03:00:00.000000,@taraaaaaaaaar,たら之ソース,8382
253,pic.twitter.com/thrc9GrXyU,[https://rts-pctr.c.yimg.jp/BgIFgYJGVIVv8u0nVJ...,2024-01-29 00:31:00.000000,@korone_suki_,びび🥐,118


In [1]:
from src import utils2
df = utils2.get_tweet('#絵かゆ',7,'holo')
df

　date_range -> 7/7 | tweets -> 240

,url,images,date,userId,userName,likeCount
0,https://twitter.com/Himawarino_seed/status/175...,[https://rts-pctr.c.yimg.jp/BgIFgYJGVIVv8u0nVJ...,2024-02-08,Himawarino_seed,🎨てんてこまい🎨,27
1,https://twitter.com/nana_77_6/status/175553143...,[https://rts-pctr.c.yimg.jp/BgIFgYJGVIVv8u0nVJ...,2024-02-08,nana_77_6,なな,41
2,https://twitter.com/Red_O7/status/175553100376...,[https://rts-pctr.c.yimg.jp/BgIFgYJGVIVv8u0nVJ...,2024-02-08,Red_O7,Red*o/れっど*,278
3,https://twitter.com/tabi66837/status/175551730...,[https://rts-pctr.c.yimg.jp/BgIFgYJGVIVv8u0nVJ...,2024-02-08,tabi66837,旅ノ道,24
4,https://twitter.com/jz0_pk/status/175551434996...,[https://rts-pctr.c.yimg.jp/BgIFgYJGVIVv8u0nVJ...,2024-02-08,jz0_pk,アラブ系男子🌈,332
...,...,...,...,...,...,...
244,https://twitter.com/nanaco_neco/status/1752894...,[https://rts-pctr.c.yimg.jp/BgIFgYJGVIVv8u0nVJ...,2024-02-01,nanaco_neco,ななこん,93
245,https://twitter.com/fuROyu_/status/17528946833...,[https://rts-pctr.c.yimg.jp/BgIFgYJGVIVv8u0nVJ...,2024-02-01,fuROyu_,ふろゆー。手描きホロライブ,74
246,https://twitter.com/kekekenokeii/status/175273...,[https://rts-pctr.c.yimg.jp/BgIFgYJGVIVv8u0nVJ...,2024-01-31,kekekenokeii,ケケケのけい,67
247,https://twitter.com/taku73268128/status/175272...,[https://rts-pctr.c.yimg.jp/BgIFgYJGVIVv8u0nVJ...,2024-01-31,taku73268128,taku7326,1451


In [1]:
from src import sqlite
sqlite.update(df , '#つのまきあーと' , 'holo')

NameError: name 'df' is not defined

In [4]:
records = sqlite.search(hashtag='#絵かゆ')
for rec in records:
    print(rec)
    print('-------------')

Hashtag: #絵かゆ
Mode: holo
URL: https://twitter.com/JR01747486/status/1755095769324118326?utm_source=yjrealtime&utm_medium=search
Date: 2024-02-07
Images: ['https://rts-pctr.c.yimg.jp/BgIFgYJGVIVv8u0nVJvw8B2FauAYMF4jOP5zFJAV4i6hjVpND1OjWZ7RVcmujne-1YYL1xhtYZHIWTdYMU8XnJfDpFAqL5hHDK0f3Jtem-i2Kasv1JEUObLoor-MXY6Cxz2f1TBbcMQZOAMxThTD5s-MamEEHQ2p_Pgqasm8FwT4JK8ibiGJEdlOsjd2S5Bo']
User ID: JR01747486
User Name: nas15ren@イラスト練習中
Like Count: 24

-------------
Hashtag: #絵かゆ
Mode: holo
URL: https://twitter.com/Ka_me22/status/1755027487195865099?utm_source=yjrealtime&utm_medium=search
Date: 2024-02-07
Images: ['https://rts-pctr.c.yimg.jp/BgIFgYJGVIVv8u0nVJvw8B2FauAYMF4jOP5zFJAV4i6hjVpND1OjWZ7RVcmujne-8cuwZ-R6-E9HMCQDQWBDXmoBZr7wKKC0etca8yPA3LgeyjlYBf7g05y-qRbOOY57d8CF8oPyQ1MBOv5eM2chco_8--Yvtp9kt7IraaOfZXu9It1PdfM4RO16eesoL4P1']
User ID: Ka_me22
User Name: カメ🐢
Like Count: 216

-------------
Hashtag: #絵かゆ
Mode: holo
URL: https://twitter.com/RamKilog/status/1755155527867486681?utm_source=yjrealtime&

In [7]:
len(set(df['url'])) , len(df['url'])

(256, 256)